In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tempfile

import tensorflow.compat.v1 as tf

from tensorflow import keras
print("TensorFlow version is ", tf.__version__)

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
base_dir = os.getcwd()
train_dir = os.path.join(base_dir, 'train_images_pre')
dev_dir = os.path.join(base_dir, 'dev_images_pre')
test_dir = os.path.join(base_dir, 'test_images_pre')

train_0_dir = os.path.join(train_dir, '0')
print ('Total training class 0 images:', len(os.listdir(train_0_dir)))

train_1_dir = os.path.join(train_dir, '1')
print ('Total training class 1 images:', len(os.listdir(train_1_dir)))

train_2_dir = os.path.join(train_dir, '2')
print ('Total training class 2 images:', len(os.listdir(train_2_dir)))

train_3_dir = os.path.join(train_dir, '3')
print ('Total training class 3 images:', len(os.listdir(train_3_dir)))

train_4_dir = os.path.join(train_dir, '4')
print ('Total training class 4 images:', len(os.listdir(train_4_dir)))

In [ ]:
dev_0_dir = os.path.join(dev_dir, '0')
print ('Total dev class 0 images:', len(os.listdir(dev_0_dir)))

dev_1_dir = os.path.join(dev_dir, '1')
print ('Total dev class 1 images:', len(os.listdir(dev_1_dir)))

dev_2_dir = os.path.join(dev_dir, '2')
print ('Total dev class 2 images:', len(os.listdir(dev_2_dir)))

dev_3_dir = os.path.join(dev_dir, '3')
print ('Total dev class 3 images:', len(os.listdir(dev_3_dir)))

dev_4_dir = os.path.join(dev_dir, '4')
print ('Total dev class 4 images:', len(os.listdir(dev_4_dir)))

In [ ]:
test_0_dir = os.path.join(test_dir, '0')
print ('Total test class 0 images:', len(os.listdir(test_0_dir)))

test_1_dir = os.path.join(test_dir, '1')
print ('Total test class 1 images:', len(os.listdir(test_1_dir)))

test_2_dir = os.path.join(test_dir, '2')
print ('Total test class 2 images:', len(os.listdir(test_2_dir)))

test_3_dir = os.path.join(test_dir, '3')
print ('Total test class 3 images:', len(os.listdir(test_3_dir)))

test_4_dir = os.path.join(test_dir, '4')
print ('Total test class 4 images:', len(os.listdir(test_4_dir)))

In [ ]:
image_size = 224
batch_size = 32

# Rescale all images by 1./255 and apply image augmentation
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255,rotation_range=360, horizontal_flip=True,vertical_flip=True)

dev_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Flow training images in batches using train_datagen generator
train_generator = train_datagen.flow_from_directory(
                train_dir,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='categorical')

# Flow dev images in batches using dev_datagen generator
dev_generator = dev_datagen.flow_from_directory(
                dev_dir, # Source directory for the dev images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode='categorical')

# Flow test images in batches using test_datagen generator
test_generator = test_datagen.flow_from_directory(
                test_dir, # Source directory for the test images
                target_size=(image_size, image_size),
                batch_size=batch_size,
                class_mode='categorical')

In [ ]:
IMG_SHAPE = (image_size, image_size, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False
base_model.summary()

In [ ]:
# Add fully connected layers
model = tf.keras.Sequential([
  base_model,
  keras.layers.GlobalAveragePooling2D(),
  keras.layers.Dense(5, activation='softmax')
])

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

model.summary()

In [ ]:
#set up callback
checkpoint_path = "Reg01/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
#train model
epochs = 1
steps_per_epoch = train_generator.n // batch_size
dev_steps = dev_generator.n // batch_size

history = model.fit_generator(train_generator,
                              steps_per_epoch = steps_per_epoch,
                              epochs=epochs,
                              workers=4,
                              validation_data=dev_generator,
                              validation_steps=dev_steps, callbacks=[cp_callback],
                              class_weight={0:641./1264., 1:641./259., 2:641./699., 3:641./135., 4:641./207.})

In [ ]:
#plot accuracy and loss
acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,max(plt.ylim())])
plt.title('Training and Validation Loss')
plt.savefig('attempt1_plots.png')
plt.show()

In [ ]:
# Fine tune by unfreezing weights
base_model.trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

model.summary()

In [ ]:
#continue to train model
history_fine = model.fit_generator(train_generator,
                              steps_per_epoch = steps_per_epoch,
                              epochs=epochs,
                              workers=4,
                              validation_data=dev_generator,
                              validation_steps=dev_steps, callbacks=[cp_callback],
                              class_weight={0:641./1264., 1:641./259., 2:641./699., 3:641./135., 4:641./207.})

In [ ]:
#plot accuracy and loss
acc = history_fine.history['categorical_accuracy']
val_acc = history_fine.history['val_categorical_accuracy']

loss = history_fine.history['loss']
val_loss = history_fine.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,max(plt.ylim())])
plt.title('Training and Validation Loss')
plt.savefig('attempt1_plots.png')
plt.show()